In [118]:
DATA_PATH='..data/Raw/survey_results_public.csv'

In [116]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import pickle

%matplotlib inline

In [93]:
#reaad data
df= pd.read_csv(DATA_PATH)

In [94]:
#print shape of the data
df.shape

(83439, 48)

In [20]:
#show first 5 entries
df.head().iloc[:,15:]

,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,Monthly,C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift,Swift,PostgreSQL;SQLite,SQLite,NaN,NaN,Laravel;Symfony,NaN,NaN,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.0
1,NaN,JavaScript;Python,NaN,PostgreSQL,NaN,NaN,NaN,Angular;Flask;Vue.js,NaN,Cordova,...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
2,NaN,Assembly;C;Python;R;Rust,Julia;Python;Rust,SQLite,SQLite,Heroku,NaN,Flask,Flask,NumPy;Pandas;TensorFlow;Torch/PyTorch,...,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above,Appropriate in length,Easy,NaN
3,Monthly,JavaScript;TypeScript,JavaScript;TypeScript,NaN,NaN,NaN,NaN,Angular;jQuery,Angular;jQuery,NaN,...,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN
4,NaN,Bash/Shell;HTML/CSS;Python;SQL,Bash/Shell;HTML/CSS;Python;SQL,Elasticsearch;PostgreSQL;Redis,Cassandra;Elasticsearch;PostgreSQL;Redis,NaN,NaN,Flask,Flask,Apache Spark;Hadoop;NumPy;Pandas,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,NaN,Appropriate in length,Easy,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ResponseId                    83439 non-null  int64  
 1   MainBranch                    83439 non-null  object 
 2   Employment                    83323 non-null  object 
 3   Country                       83439 non-null  object 
 4   US_State                      14920 non-null  object 
 5   UK_Country                    4418 non-null   object 
 6   EdLevel                       83126 non-null  object 
 7   Age1stCode                    83243 non-null  object 
 8   LearnCode                     82963 non-null  object 
 9   YearsCode                     81641 non-null  object 
 10  YearsCodePro                  61216 non-null  object 
 11  DevType                       66484 non-null  object 
 12  OrgSize                       60726 non-null  object 
 13  C

In [7]:
#visualize a sample
df.sample(1).iloc[0]

ResponseId                                                                  33516
MainBranch                                         I am a developer by profession
Employment                                     Not employed, but looking for work
Country                                                                Cape Verde
US_State                                                                      NaN
UK_Country                                                                    NaN
EdLevel                              Bachelor’s degree (B.A., B.S., B.Eng., etc.)
Age1stCode                                                          11 - 17 years
LearnCode                       Other online resources (ex: videos, blogs, etc...
YearsCode                                                                       5
YearsCodePro                                                                  NaN
DevType                         Developer, mobile;Developer, front-end;Develop...
OrgSize         

__________________

In [41]:
df_comp = df['Country'].value_counts()
df_country = pd.DataFrame({'Country':df_comp.index, 'count': df_comp.values})
df_country.head()

,Country,count
0,United States of America,15288
1,India,10511
2,Germany,5625
3,United Kingdom of Great Britain and Northern I...,4475
4,Canada,3012


In [42]:
# sns.countplot(df['Country'])
px.bar(data_frame = df_country, y = 'Country', x = 'count')

## data investigation

## Data problems
### 1. wrong dtypes in columns ['YearsCodePro', 'YearsCode', 'Age1stCode'] dute to string in cols
### 2. cols to be splitted on ; ['DevType', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
###       'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith'] it has other cols so we need to do it programatically
### 3. fix EDlevel remove paranthesis and rename categories
### 4. fix Learncode 
### 5. 'Accessibility' should we keep it or categorize it (have disability yes/no)
### 6. US_state, UK_state should we keep them or add another category ex: none
### 7. nulls
___
## irrelevant cols
### 'ResponseId',  'MentalHealth', 'SurveyLength', 'SurveyEase', 'Age1stCode', 'Sexuality', 'Ethnicity'
### 'Trans', 'Gender' , 'Currency', 'OrgSize','CompFreq', 'LanguageWantToWorkWith', 'DatabaseWantToWorkWith', --> need more exploration before remove

___

## preprocessing

In [9]:
# check data types
# select relavent features
# preprocessing 

#handle wrong data types ---> 'YearsCode','YearsCodePro'

#handle strings with ; to be list of strings

### 1- handle first problem wrong dtypes of columns

In [46]:
replacements = {'YearsCodePro': {'More than 50 years':51,'Less than 1 year':0},\
               'YearsCode' : {'More than 50 years':51,'Less than 1 year':0},\
               }

In [51]:
for col , replacement in replacements.items():
    df[col] = df[col].replace(replacement).astype('float')

In [53]:
df.head().iloc[:,10:]

,YearsCodePro,DevType,OrgSize,Currency,CompTotal,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,NaN,"Developer, mobile",20 to 99 employees,EUR European Euro,4800.0,Monthly,C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift,Swift,PostgreSQL;SQLite,SQLite,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.0
1,NaN,NaN,NaN,NaN,NaN,NaN,JavaScript;Python,NaN,PostgreSQL,NaN,...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Assembly;C;Python;R;Rust,Julia;Python;Rust,SQLite,SQLite,...,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above,Appropriate in length,Easy,NaN
3,NaN,"Developer, front-end",100 to 499 employees,EUR European Euro,NaN,Monthly,JavaScript;TypeScript,JavaScript;TypeScript,NaN,NaN,...,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN
4,10.0,"Developer, desktop or enterprise applications;...","Just me - I am a freelancer, sole proprietor, ...",GBP\tPound sterling,NaN,NaN,Bash/Shell;HTML/CSS;Python;SQL,Bash/Shell;HTML/CSS;Python;SQL,Elasticsearch;PostgreSQL;Redis,Cassandra;Elasticsearch;PostgreSQL;Redis,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,NaN,Appropriate in length,Easy,NaN


### 2. split data separated by ; as list

In [99]:
def split_data(df_col, delimeter = ';'):
    '''
    split the data in data frame seires (column) on specific deleimeter and store them in list
    
    params:
        df_col: (pd.series) series contains data to be splitted
        delimeter: default ';' to split data on
    return:
        df_series: (pd.series) splitted series and the dtype is list
    '''
    # check if the data is splitable (contains the delimeter)
    def is_splitable(pd_series, delimeter = ';'):
        return pd_series.str.contains(delimeter).any()
    
    # spliting function 
    def split(pd_series , delimeter = ';'):
        return pd_series.apply(lambda x: x.split(delimeter) if type(x) != float else [])
    
    # performing split
    if is_splitable(df_col, delimeter):
        return split(df_col, delimeter)
    else:
        return df_col
        

In [105]:
for col in df.select_dtypes(include = 'object').columns:
    df[col] = split_data(df[col])
    

In [106]:
df.head().iloc[:,10:]

,YearsCodePro,DevType,OrgSize,Currency,CompTotal,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,NaN,"[Developer, mobile]",20 to 99 employees,EUR European Euro,4800.0,Monthly,"[C++, HTML/CSS, JavaScript, Objective-C, PHP, ...",[Swift],"[PostgreSQL, SQLite]",[SQLite],...,25-34 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[None of the above],[None of the above],Appropriate in length,Easy,62268.0
1,NaN,[],NaN,NaN,NaN,NaN,"[JavaScript, Python]",[],[PostgreSQL],[],...,18-24 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[None of the above],[None of the above],Appropriate in length,Easy,NaN
2,NaN,[],NaN,NaN,NaN,NaN,"[Assembly, C, Python, R, Rust]","[Julia, Python, Rust]",[SQLite],[SQLite],...,18-24 years old,[Man],No,[Prefer not to say],[Prefer not to say],[None of the above],[None of the above],Appropriate in length,Easy,NaN
3,NaN,"[Developer, front-end]",100 to 499 employees,EUR European Euro,NaN,Monthly,"[JavaScript, TypeScript]","[JavaScript, TypeScript]",[],[],...,35-44 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[I am deaf / hard of hearing],[],Appropriate in length,Neither easy nor difficult,NaN
4,10,"[Developer, desktop or enterprise applications...","Just me - I am a freelancer, sole proprietor, ...",GBP\tPound sterling,NaN,NaN,"[Bash/Shell, HTML/CSS, Python, SQL]","[Bash/Shell, HTML/CSS, Python, SQL]","[Elasticsearch, PostgreSQL, Redis]","[Cassandra, Elasticsearch, PostgreSQL, Redis]",...,25-34 years old,[Man],No,[],[White or of European descent],[None of the above],[],Appropriate in length,Easy,NaN


### 3- handle edlevel column

In [108]:
df['EdLevel'].unique()

array(['Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
       'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
       'Other doctoral degree (Ph.D., Ed.D., etc.)',
       'Some college/university study without earning a degree',
       'Something else', 'Professional degree (JD, MD, etc.)',
       'Primary/elementary school', 'Associate degree (A.A., A.S., etc.)',
       nan], dtype=object)

In [113]:
edu_dict = {'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 'Bachelor-degree',
            'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)':'Master-degree',
            'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)':'secondary-school',
            'Other doctoral degree (Ph.D., Ed.D., etc.)':'doctoral-degree',
            'Primary/elementary school':'Primary/elementary-school',
            'Associate degree (A.A., A.S., etc.)':'Associate-degree',
            'Professional degree (JD, MD, etc.)':'Professional-degree',
            'Something else':'Something-else',
            'Some college/university study without earning a degree':'college-study-without-earning-degree'
           }

In [115]:
df['EdLevel'] = df['EdLevel'].replace(edu_dict)

###  save file as pickle

In [117]:
export_path = '../data/processed/1_preprocessed.pkl'
df.to_pickle(export_path)

___

In [45]:
# # cols that has 5% or more nans 
# null_cols = [c for c in df.isnull().sum().index if df.isnull().sum()[c]/df.shape[0] > .05  ]
# null_cols

In [22]:
# #filling nulls 
# for col in test.columns:
#     if test[col].dtype == 'object':
#         test[col].fillna(test[col].mode().iloc[0], inplace = True)
#     else:
#         test[col].fillna(test[col].median(), inplace = True)

In [25]:
questionable_cols = ['YearsCodePro', 'YearsCode', 'Age1stCode']

for col in questionable_cols: 
    print(col)
    print(df[col].unique().tolist())
    print('--------------------------')
    print()

YearsCodePro
[nan, '10', '4', '5', '6', '2', '30', '9', '18', '12', '21', '1', '16', 'Less than 1 year', '15', '3', '35', '7', '8', '17', '14', '26', '25', '20', '50', '34', '11', '24', '22', '13', '31', '23', '39', '41', '27', '28', '19', '33', 'More than 50 years', '37', '29', '32', '43', '40', '38', '45', '42', '46', '36', '44', '47', '48', '49']
--------------------------

YearsCode
[nan, '7', '17', '3', '4', '6', '16', '12', '15', '10', '40', '9', '26', '14', '39', '20', '8', '19', '5', 'Less than 1 year', '22', '2', '1', '34', '21', '13', '25', '24', '30', '31', '18', '38', 'More than 50 years', '27', '41', '42', '35', '23', '28', '11', '37', '44', '43', '36', '33', '45', '29', '50', '46', '32', '47', '49', '48']
--------------------------

Age1stCode
['18 - 24 years', '11 - 17 years', '5 - 10 years', '25 - 34 years', '35 - 44 years', 'Younger than 5 years', '45 - 54 years', '55 - 64 years', nan, 'Older than 64 years']
--------------------------



In [ ]:
df.replace()

In [ ]:
###########################3relevant features:
# ResponseId                                                                 
# MainBranch                                        
# Employment                                                     
# Country                                                                   
# EdLevel                              
# Age1stCode                                                          
# LearnCode                                                                  
# YearsCode                                                                      
# YearsCodePro                                                                    
# DevType                         
# Currency                                                    
# CompTotal                                                               
# CompFreq                                                                   
# LanguageHaveWorkedWith                                                
# LanguageWantToWorkWith                                                   
# DatabaseHaveWorkedWith                               
# DatabaseWantToWorkWith                                                     
# PlatformHaveWorkedWith                                                        
# PlatformWantToWorkWith                                                       
# WebframeHaveWorkedWith                                                        
# WebframeWantToWorkWith                                                       
# MiscTechHaveWorkedWith                                                        
# MiscTechWantToWorkWith                                                       
# ToolsTechHaveWorkedWith                                                       
# ToolsTechWantToWorkWith                                                       
# NEWCollabToolsHaveWorkedWith             
# NEWCollabToolsWantToWorkWith                       
# OpSys                                                                     

# NEWStuck                        
# NEWSOSites                                          

# SOVisitFreq                                                 
# SOAccount                                                                     
# SOPartFreq                                    
# SOComm                                                             

# Age                                                              
# Gender                                                                       

                                                                
